# Black-Scholes Option Greeks

This notebook implements and visualises the **Black-Scholes option Greeks** from scratch — the sensitivity measures that quantify how an option's price changes with respect to market inputs.

### Greeks covered
| Greek | Measures sensitivity to... |
|---|---|
| **Delta** (Δ) | Changes in the underlying asset price |
| **Gamma** (Γ) | Rate of change of Delta |
| **Theta** (Θ) | Time decay (passage of time) |

All formulas derive from the Black-Scholes model for European call options.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

## Black-Scholes Formulas

For a European call option with underlying price *S*, strike *K*, time to expiry *T*, risk-free rate *r*, and volatility *σ*:

$$d_1 = \frac{\ln(S/K) + (r + \frac{1}{2}\sigma^2)T}{\sigma\sqrt{T}}, \quad d_2 = d_1 - \sigma\sqrt{T}$$

$$\Delta = N(d_1), \quad \Gamma = \frac{N'(d_1)}{S\sigma\sqrt{T}}, \quad \Theta = -\frac{S\sigma N'(d_1)}{2\sqrt{T}} - rKe^{-rT}N(d_2)$$

In [ ]:
def d1(S, K, T, r, sigma):
    return (np.log(S / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))

def d2(S, K, T, r, sigma):
    return d1(S, K, T, r, sigma) - sigma * np.sqrt(T)

def call_delta(S, K, T, r, sigma):
    """Delta: probability (risk-neutral) that the option expires in-the-money."""
    return norm.cdf(d1(S, K, T, r, sigma))

def call_gamma(S, K, T, r, sigma):
    """Gamma: rate of change of Delta — highest at-the-money."""
    return norm.pdf(d1(S, K, T, r, sigma)) / (S * sigma * np.sqrt(T))

def call_theta(S, K, T, r, sigma):
    """Theta: daily time decay (negative — option loses value as expiry approaches)."""
    _d1 = d1(S, K, T, r, sigma)
    _d2 = d2(S, K, T, r, sigma)
    return (
        -S * norm.pdf(_d1) * sigma / (2 * np.sqrt(T))
        - r * K * np.exp(-r * T) * norm.cdf(_d2)
    )

## 1. Delta and Gamma vs Underlying Price

Delta ranges from 0 (deep out-of-the-money) to 1 (deep in-the-money). Gamma peaks at-the-money, where Delta is most sensitive to price moves.

In [ ]:
K     = 100      # Strike price
T     = 30 / 365 # 30 days to expiry
r     = 0.05     # Risk-free rate
sigma = 0.2      # Volatility (20%)

prices = np.linspace(70, 130, 300)

deltas = [call_delta(S, K, T, r, sigma) for S in prices]
gammas = [call_gamma(S, K, T, r, sigma) for S in prices]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
fig.suptitle('Black-Scholes Greeks: 30-Day Call Option', fontsize=14, fontweight='bold')

ax1.plot(prices, deltas, color='steelblue', linewidth=2)
ax1.axvline(K, color='gray', linestyle='--', linewidth=1, label=f'Strike = {K}')
ax1.axhline(0.5, color='tomato', linestyle=':', linewidth=1, label='Delta = 0.5 (ATM)')
ax1.set_title('Delta (Δ)')
ax1.set_xlabel('Underlying Price ($)')
ax1.set_ylabel('Delta')
ax1.legend()
ax1.grid(alpha=0.3)

ax2.plot(prices, gammas, color='green', linewidth=2)
ax2.axvline(K, color='gray', linestyle='--', linewidth=1, label=f'Strike = {K}')
ax2.set_title('Gamma (Γ)')
ax2.set_xlabel('Underlying Price ($)')
ax2.set_ylabel('Gamma')
ax2.legend()
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 2. Theta vs Underlying Price — Effect of Time to Expiry

Theta is always negative for long calls. Time decay accelerates as the option approaches expiry, especially at-the-money.

In [ ]:
T_values   = [0.1, 0.5, 1.0]          # Time to maturity (years)
colors     = ['tomato', 'steelblue', 'green']
broad_prices = np.linspace(50, 150, 300)

fig, ax = plt.subplots(figsize=(10, 6))

for T_val, color in zip(T_values, colors):
    thetas = [call_theta(S, K, T_val, r, sigma) for S in broad_prices]
    ax.plot(broad_prices, thetas, color=color, linewidth=2, label=f'T = {T_val} yr')

ax.axhline(0, color='black', linewidth=0.8, linestyle='--')
ax.axvline(K, color='gray', linestyle=':', linewidth=1, label=f'Strike = {K}')
ax.set_title('Theta (Θ) at Different Times to Expiry', fontsize=13)
ax.set_xlabel('Underlying Asset Price ($)')
ax.set_ylabel('Theta ($/day)')
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 3. All Three Greeks on One Chart

Overlaying normalised Greeks to show their relationship at a given expiry.

In [ ]:
T_fixed = 30 / 365

deltas_plot = np.array([call_delta(S, K, T_fixed, r, sigma) for S in prices])
gammas_plot = np.array([call_gamma(S, K, T_fixed, r, sigma) for S in prices])
thetas_plot = np.array([call_theta(S, K, T_fixed, r, sigma) for S in prices])

fig, ax = plt.subplots(figsize=(11, 6))

ax.plot(prices, deltas_plot,  color='steelblue', linewidth=2, label='Delta (Δ)')
ax.plot(prices, gammas_plot,  color='green',     linewidth=2, label='Gamma (Γ)')
ax.plot(prices, thetas_plot,  color='tomato',    linewidth=2, label='Theta (Θ)')
ax.axvline(K, color='gray', linestyle='--', linewidth=1, label=f'Strike K = {K}')
ax.axhline(0, color='black', linewidth=0.8, linestyle=':')

ax.set_title(f'Option Greeks — 30-Day Call (K={K}, σ={sigma}, r={r})', fontsize=13)
ax.set_xlabel('Underlying Price ($)')
ax.set_ylabel('Greek Value')
ax.legend()
ax.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# Summary at-the-money
atm_idx = np.argmin(np.abs(prices - K))
print("At-the-money values (S = K = 100):")
print(f"  Delta : {deltas_plot[atm_idx]:.4f}")
print(f"  Gamma : {gammas_plot[atm_idx]:.4f}")
print(f"  Theta : {thetas_plot[atm_idx]:.4f} $/day")

## Key Insights

- **Delta** is roughly 0.5 at-the-money and approaches 1 deep in-the-money — it behaves like a hedge ratio
- **Gamma** is highest at-the-money, meaning Delta is most unstable there (gamma risk)
- **Theta** is most negative at-the-money and for shorter maturities — time decay is fastest near expiry
- The Gamma-Theta relationship is a core tension in options trading: high Gamma (good for dynamic hedging) always comes with high Theta (costly time decay)